# <font color='red'>Modelisation :</font>


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from scipy import stats
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, AdaBoostRegressor
import xgboost as xgb
import shap
from sklearn.tree import DecisionTreeRegressor

# Pour afficher les graphiques directement dans le notebook
%matplotlib inline



In [ ]:
# Nous repartons des données préalablement nettoyées et enregistrées (notebook Preprocessing)
df = pd.read_csv('data_cleaned.csv',index_col=[0])
df_thermique = pd.read_csv('data_thermique.csv',index_col=[0])
df_hybride = pd.read_csv('data_hybride.csv',index_col=[0])

### <font color='blue'> Séparation train & test  des 3 datasets + standardisation: </font>

In [ ]:
#scinder les data en X et y : 
X = df.drop("Emission_co2_(g/km)",axis = 1)
y = df["Emission_co2_(g/km)"]

y_thermique = df_thermique['Emission_co2_(g/km)']
X_thermique = df_thermique.drop('Emission_co2_(g/km)', axis=1)


y_hybride = df_hybride['Emission_co2_(g/km)']
X_hybride = df_hybride.drop('Emission_co2_(g/km)', axis=1)

#scinder les datas en train et test : 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

X_thermique_train, X_thermique_test, y_thermique_train, y_thermique_test = train_test_split(X_thermique, y_thermique, test_size=0.3, random_state=42)

X_hybride_train, X_hybride_test, y_hybride_train, y_hybride_test = train_test_split(X_hybride, y_hybride, test_size=0.3, random_state=42)

#Standardiser : 
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_thermique_train_scaled = scaler.fit_transform(X_thermique_train)
X_thermique_test_scaled = scaler.transform(X_thermique_test)

X_hybride_train_scaled = scaler.fit_transform(X_hybride_train)
X_hybride_test_scaled = scaler.transform(X_hybride_test)


### <font color='blue'> Régression lineaire: </font>

Avant de construire un modèle de régression linéaire, il est crucial de vérifier si la variable cible suit une distribution normale, car cela affecte la validité des résultats et l'interprétation des coefficients. L'analyse de la normalité garantit que les hypothèses de la régression linéaire sont satisfaites, renforçant ainsi la fiabilité et la pertinence du modèle.

In [ ]:
# Analyser la normalité de la variable cible dans la data complète
plt.figure(figsize=(12, 6))
sns.histplot(y, kde=True, color="skyblue", edgecolor="black", linewidth=1, bins=30)
plt.title('Histogramme de CO2 (g/km) ', fontsize=16)
plt.xlabel('Valeurs de la variable cible', fontsize=14)
plt.ylabel('Fréquence', fontsize=14)

# Enregistrer l'image
plt.savefig('histogramme_variable_cible.png', bbox_inches='tight')


### <font color='blue'> Analyse data voitures thermiques : </font>

In [ ]:
# Tracer un histogramme
sns.histplot(df_thermique['Emission_co2_(g/km)'], kde=True, bins=50, edgecolor='black', color='blue', line_kws={'color': 'red', 'linewidth': 2})


# Calculer la moyenne et la médiane
mean_value = df_thermique['Emission_co2_(g/km)'].mean()
median_value = df_thermique['Emission_co2_(g/km)'].median()

# Tracer des lignes verticales pour représenter la moyenne et la médiane
plt.axvline(mean_value, color='red', linewidth=1, label='Moyenne de distribution')
plt.axvline(median_value, color='red', linestyle='dashed', linewidth=1, label='Médiane de distribution')

# Ajouter une légende
plt.legend()

# Afficher le graphique
plt.show()


### <font color='blue'> Sans validation croisée : </font>

In [ ]:
# Création et entraînement du modèle de régression linéaire
model = LinearRegression()
model.fit(X_thermique_train_scaled, y_thermique_train)

# Prédiction des valeurs cibles pour les ensembles d'entraînement et de test
y_thermique_train_pred = model.predict(X_thermique_train_scaled)
y_thermique_test_pred = model.predict(X_thermique_test_scaled)

# Calcul des métriques pour l'entraînement
r2_train = r2_score(y_thermique_train, y_thermique_train_pred)
mae_train = mean_absolute_error(y_thermique_train, y_thermique_train_pred)
rmse_train = np.sqrt(mean_squared_error(y_thermique_train, y_thermique_train_pred))

# Calcul des métriques pour le test
r2_test = r2_score(y_thermique_test, y_thermique_test_pred)
mae_test = mean_absolute_error(y_thermique_test, y_thermique_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_thermique_test, y_thermique_test_pred))

print("Régression Linéaire sans validation croisée")
print(f"R² pour l'entraînement : {r2_train}")
print(f"R² pour le test : {r2_test}")
print(f"MAE pour l'entraînement : {mae_train}")
print(f"MAE pour le test : {mae_test}")
print(f"RMSE pour l'entraînement : {rmse_train}")
print(f"RMSE pour le test : {rmse_test}")


### <font color='blue'> Avec validation croisée : </font>

In [ ]:
cv_results = cross_validate(model, X_thermique_train_scaled, y_thermique_train, cv=10,
                            scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'),
                            return_train_score=True)

# Moyennes des scores R², MAE et RMSE pour l'entraînement et le test
mean_r2_train = np.mean(cv_results['train_r2'])
mean_r2_test = np.mean(cv_results['test_r2'])
mean_mae_train = -np.mean(cv_results['train_neg_mean_absolute_error'])
mean_mae_test = -np.mean(cv_results['test_neg_mean_absolute_error'])
mean_rmse_train = -np.mean(cv_results['train_neg_root_mean_squared_error'])
mean_rmse_test = -np.mean(cv_results['test_neg_root_mean_squared_error'])

print("Régression Linéaire avec validation croisée")
print(f"R² moyen pour l'entraînement : {mean_r2_train}")
print(f"R² moyen pour le test : {mean_r2_test}")
print(f"MAE moyen pour l'entraînement : {mean_mae_train}")
print(f"MAE moyen pour le test : {mean_mae_test}")
print(f"RMSE moyen pour l'entraînement : {mean_rmse_train}")
print(f"RMSE moyen pour le test : {mean_rmse_test}")



### <font color='blue'> Analyse des residus : </font>

In [ ]:
# Calculer les résidus
residuals = y_thermique_test - y_thermique_test_pred

# Visualiser les résidus
plt.figure(figsize=(6, 6))
sns.histplot(residuals, kde=True)
plt.title('Distribution des résidus')
plt.xlabel('Résidus')
plt.ylabel('Fréquence')
plt.show()
# Enregistrer l'image
plt.savefig('distribution des lr thermiques.png', bbox_inches='tight')

# Vérifier l'homoscédasticité
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_thermique_test_pred, y=residuals)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Résidus par rapport aux valeurs prédites')
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.show()


In [ ]:
# Calculer les résidus normalisés
residuals_standardized = (residuals - residuals.mean()) / residuals.std()

# Q-Q plot
plt.figure(figsize=(8, 6))
stats.probplot(residuals_standardized, dist="norm", plot=plt)
plt.title("Q-Q Plot des Résidus")
plt.xlabel("Quantiles théoriques (distribution normale)")
plt.ylabel("Quantiles observés")
plt.show()


In [ ]:

# Créer le nuage de points entre les valeurs prédites et les valeurs réelles avec une ligne de corrélation
plt.figure(figsize=(10, 6))
plt.scatter(y_thermique_test_pred, y_thermique_test, alpha=0.5)
plt.plot([y_thermique_test.min(), y_thermique_test.max()], [y_thermique_test.min(), y_thermique_test.max()], 'r--', lw=2)  # Ajouter une ligne de corrélation parfaite
plt.title("Nuage de points entre les valeurs prédites et les valeurs réelles (test)")
plt.xlabel("Valeurs prédites")
plt.ylabel("Valeurs réelles")
plt.grid(True)
plt.show()



In [ ]:

# Définir le style de Seaborn sur darkgrid
sns.set_style("darkgrid")

# Créer la boîte à moustaches des résidus avec Seaborn
plt.figure(figsize=(10, 6))
sns.boxplot(x=residuals, orient='h', color='cornflowerblue')
plt.title("Boîte à moustaches des résidus")
plt.xlabel("Résidus")
plt.grid(True)
plt.show()



#### <font color='pink'> Elasticnet / Ridge et lasso pour les voitures thermiques : </font>

In [ ]:
# Initialiser les modèles
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)
ridge = Ridge(alpha=0.1)
lasso = Lasso(alpha=0.1)

# Entraîner les modèles
elastic_net.fit(X_thermique_train_scaled, y_thermique_train)
ridge.fit(X_thermique_train_scaled, y_thermique_train)
lasso.fit(X_thermique_train_scaled, y_thermique_train)

# Prédire les valeurs cibles pour les ensembles d'entraînement et de test
y_train_pred_elastic_net = elastic_net.predict(X_thermique_train_scaled)
y_test_pred_elastic_net = elastic_net.predict(X_thermique_test_scaled)

y_train_pred_ridge = ridge.predict(X_thermique_train_scaled)
y_test_pred_ridge = ridge.predict(X_thermique_test_scaled)

y_train_pred_lasso = lasso.predict(X_thermique_train_scaled)
y_test_pred_lasso = lasso.predict(X_thermique_test_scaled)

# Calculer les scores R² pour l'entraînement et le test
r2_train_elastic_net = r2_score(y_thermique_train, y_train_pred_elastic_net)
r2_test_elastic_net = r2_score(y_thermique_test, y_test_pred_elastic_net)

r2_train_ridge = r2_score(y_thermique_train, y_train_pred_ridge)
r2_test_ridge = r2_score(y_thermique_test, y_test_pred_ridge)

r2_train_lasso = r2_score(y_thermique_train, y_train_pred_lasso)
r2_test_lasso = r2_score(y_thermique_test, y_test_pred_lasso)

# Calculer les erreurs (MAE et RMSE) pour l'entraînement et le test
mae_train_elastic_net = mean_absolute_error(y_thermique_train, y_train_pred_elastic_net)
rmse_train_elastic_net = mean_squared_error(y_thermique_train, y_train_pred_elastic_net, squared=False)
mae_test_elastic_net = mean_absolute_error(y_thermique_test, y_test_pred_elastic_net)
rmse_test_elastic_net = mean_squared_error(y_thermique_test, y_test_pred_elastic_net, squared=False)

mae_train_ridge = mean_absolute_error(y_thermique_train, y_train_pred_ridge)
rmse_train_ridge = mean_squared_error(y_thermique_train, y_train_pred_ridge, squared=False)
mae_test_ridge = mean_absolute_error(y_thermique_test, y_test_pred_ridge)
rmse_test_ridge = mean_squared_error(y_thermique_test, y_test_pred_ridge, squared=False)

mae_train_lasso = mean_absolute_error(y_thermique_train, y_train_pred_lasso)
rmse_train_lasso = mean_squared_error(y_thermique_train, y_train_pred_lasso, squared=False)
mae_test_lasso = mean_absolute_error(y_thermique_test, y_test_pred_lasso)
rmse_test_lasso = mean_squared_error(y_thermique_test, y_test_pred_lasso, squared=False)

# Afficher les résultats
print("ElasticNet - R² pour l'entraînement:", r2_train_elastic_net)
print("ElasticNet - R² pour le test:", r2_test_elastic_net)
print("ElasticNet - Mean Absolute Error (MAE) pour l'entraînement:", mae_train_elastic_net)
print("ElasticNet - Mean Absolute Error (MAE) pour le test:", mae_test_elastic_net)
print("ElasticNet - Root Mean Squared Error (RMSE) pour l'entraînement:", rmse_train_elastic_net)
print("ElasticNet - Root Mean Squared Error (RMSE) pour le test:", rmse_test_elastic_net)

print("\nRidge - R² pour l'entraînement:", r2_train_ridge)
print("Ridge - R² pour le test:", r2_test_ridge)
print("Ridge - Mean Absolute Error (MAE) pour l'entraînement:", mae_train_ridge)
print("Ridge - Mean Absolute Error (MAE) pour le test:", mae_test_ridge)
print("Ridge - Root Mean Squared Error (RMSE) pour l'entraînement:", rmse_train_ridge)
print("Ridge - Root Mean Squared Error (RMSE) pour le test:", rmse_test_ridge)

print("\nLasso - R² pour l'entraînement:", r2_train_lasso)
print("Lasso - R² pour le test:", r2_test_lasso)
print("Lasso - Mean Absolute Error (MAE) pour l'entraînement:", mae_train_lasso)
print("Lasso - Mean Absolute Error (MAE) pour le test:", mae_test_lasso)
print("Lasso - Root Mean Squared Error (RMSE) pour l'entraînement:", rmse_train_lasso)
print("Lasso - Root Mean Squared Error (RMSE) pour le test:", rmse_test_lasso)


### <font color='blue'> Analyse data voitures hybrides : </font>

In [ ]:
# Tracer un histogramme
sns.histplot(df_hybride['Emission_co2_(g/km)'], kde=True, bins=50, edgecolor='black', color='blue', line_kws={'color': 'red', 'linewidth': 2})


# Calculer la moyenne et la médiane
mean_value = df_hybride['Emission_co2_(g/km)'].mean()
median_value = df_hybride['Emission_co2_(g/km)'].median()

# Tracer des lignes verticales pour représenter la moyenne et la médiane
plt.axvline(mean_value, color='red', linewidth=1, label='Moyenne de distribution')
plt.axvline(median_value, color='red', linestyle='dashed', linewidth=1, label='Médiane de distribution')

# Ajouter une légende
plt.legend()

# Afficher le graphique
plt.show()


### <font color='blue'> sans validation croisée : </font>

In [ ]:
# Création et entraînement du modèle de régression linéaire
model = LinearRegression()
model.fit(X_hybride_train_scaled, y_hybride_train)

# Prédiction des valeurs cibles pour les ensembles d'entraînement et de test
y_hybride_train_pred = model.predict(X_hybride_train_scaled)
y_hybride_test_pred = model.predict(X_hybride_test_scaled)

# Calcul des métriques pour l'entraînement
r2_train = r2_score(y_hybride_train, y_hybride_train_pred)
mae_train = mean_absolute_error(y_hybride_train, y_hybride_train_pred)
rmse_train = np.sqrt(mean_squared_error(y_hybride_train, y_hybride_train_pred))

# Calcul des métriques pour le test
r2_test = r2_score(y_hybride_test, y_hybride_test_pred)
mae_test = mean_absolute_error(y_hybride_test, y_hybride_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_hybride_test, y_hybride_test_pred))

print("Régression Linéaire sans validation croisée")
print(f"R² pour l'entraînement : {r2_train}")
print(f"R² pour le test : {r2_test}")
print(f"MAE pour l'entraînement : {mae_train}")
print(f"MAE pour le test : {mae_test}")
print(f"RMSE pour l'entraînement : {rmse_train}")
print(f"RMSE pour le test : {rmse_test}")


### <font color='blue'> Avec validation croisée : </font>

In [ ]:
# Validation croisée
cv_results = cross_validate(model, X_hybride_train_scaled, y_hybride_train, cv=10,
                            scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'),
                            return_train_score=True)

# Moyennes des scores R², MAE et RMSE pour l'entraînement et le test
mean_r2_train = np.mean(cv_results['train_r2'])
mean_r2_test = np.mean(cv_results['test_r2'])
mean_mae_train = -np.mean(cv_results['train_neg_mean_absolute_error'])
mean_mae_test = -np.mean(cv_results['test_neg_mean_absolute_error'])
mean_rmse_train = -np.mean(cv_results['train_neg_root_mean_squared_error'])
mean_rmse_test = -np.mean(cv_results['test_neg_root_mean_squared_error'])

print("Régression Linéaire avec validation croisée")
print(f"R² moyen pour l'entraînement : {mean_r2_train}")
print(f"R² moyen pour le test : {mean_r2_test}")
print(f"MAE moyen pour l'entraînement : {mean_mae_train}")
print(f"MAE moyen pour le test : {mean_mae_test}")
print(f"RMSE moyen pour l'entraînement : {mean_rmse_train}")
print(f"RMSE moyen pour le test : {mean_rmse_test}")


### <font color='blue'> Analyse des résidus : </font>

In [ ]:
# Calculer les résidus
residuals = y_hybride_test - y_hybride_test_pred

# Visualiser les résidus
plt.figure(figsize=(6, 6))
sns.histplot(residuals, kde=True)
plt.title('Distribution des résidus')
plt.xlabel('Résidus')
plt.ylabel('Fréquence')
plt.show()
# Enregistrer l'image
plt.savefig('distribution des lr thermiques.png', bbox_inches='tight')

# Vérifier l'homoscédasticité
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_hybride_test_pred, y=residuals)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Résidus par rapport aux valeurs prédites')
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.show()


#### <font color='pink'> Elasticnet / Ridge et lasso pour les voitures hybrides : </font>

In [ ]:
# Initialiser les modèles
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)
ridge = Ridge(alpha=0.1)
lasso = Lasso(alpha=0.1)

# Entraîner les modèles
elastic_net.fit(X_hybride_train_scaled, y_hybride_train)
ridge.fit(X_hybride_train_scaled, y_hybride_train)
lasso.fit(X_hybride_train_scaled, y_hybride_train)

# Prédire les valeurs cibles pour les ensembles d'entraînement et de test
y_train_pred_elastic_net = elastic_net.predict(X_hybride_train_scaled)
y_test_pred_elastic_net = elastic_net.predict(X_hybride_test_scaled)

y_train_pred_ridge = ridge.predict(X_hybride_train_scaled)
y_test_pred_ridge = ridge.predict(X_hybride_test_scaled)

y_train_pred_lasso = lasso.predict(X_hybride_train_scaled)
y_test_pred_lasso = lasso.predict(X_hybride_test_scaled)


# Calculer les scores R² pour l'entraînement et le test
r2_train_elastic_net = r2_score(y_hybride_train, y_train_pred_elastic_net)
r2_test_elastic_net = r2_score(y_hybride_test, y_test_pred_elastic_net)

r2_train_ridge = r2_score(y_hybride_train, y_train_pred_ridge)
r2_test_ridge = r2_score(y_hybride_test, y_test_pred_ridge)

r2_train_lasso = r2_score(y_hybride_train, y_train_pred_lasso)
r2_test_lasso = r2_score(y_hybride_test, y_test_pred_lasso)

# Calculer les erreurs (MAE et RMSE) pour l'entraînement et le test
mae_train_elastic_net = mean_absolute_error(y_hybride_train, y_train_pred_elastic_net)
rmse_train_elastic_net = mean_squared_error(y_hybride_train, y_train_pred_elastic_net, squared=False)
mae_test_elastic_net = mean_absolute_error(y_hybride_test, y_test_pred_elastic_net)
rmse_test_elastic_net = mean_squared_error(y_hybride_test, y_test_pred_elastic_net, squared=False)

mae_train_ridge = mean_absolute_error(y_hybride_train, y_train_pred_ridge)
rmse_train_ridge = mean_squared_error(y_hybride_train, y_train_pred_ridge, squared=False)
mae_test_ridge = mean_absolute_error(y_hybride_test, y_test_pred_ridge)
rmse_test_ridge = mean_squared_error(y_hybride_test, y_test_pred_ridge, squared=False)

mae_train_lasso = mean_absolute_error(y_hybride_train, y_train_pred_lasso)
rmse_train_lasso = mean_squared_error(y_hybride_train, y_train_pred_lasso, squared=False)
mae_test_lasso = mean_absolute_error(y_hybride_test, y_test_pred_lasso)
rmse_test_lasso = mean_squared_error(y_hybride_test, y_test_pred_lasso, squared=False)

# Afficher les résultats
print("ElasticNet - R² pour l'entraînement:", r2_train_elastic_net)
print("ElasticNet - R² pour le test:", r2_test_elastic_net)
print("ElasticNet - Mean Absolute Error (MAE) pour l'entraînement:", mae_train_elastic_net)
print("ElasticNet - Mean Absolute Error (MAE) pour le test:", mae_test_elastic_net)
print("ElasticNet - Root Mean Squared Error (RMSE) pour l'entraînement:", rmse_train_elastic_net)
print("ElasticNet - Root Mean Squared Error (RMSE) pour le test:", rmse_test_elastic_net)

print("\nRidge - R² pour l'entraînement:", r2_train_ridge)
print("Ridge - R² pour le test:", r2_test_ridge)
print("Ridge - Mean Absolute Error (MAE) pour l'entraînement:", mae_train_ridge)
print("Ridge - Mean Absolute Error (MAE) pour le test:", mae_test_ridge)
print("Ridge - Root Mean Squared Error (RMSE) pour l'entraînement:", rmse_train_ridge)
print("Ridge - Root Mean Squared Error (RMSE) pour le test:", rmse_test_ridge)

print("\nLasso - R² pour l'entraînement:", r2_train_lasso)
print("Lasso - R² pour le test:", r2_test_lasso)
print("Lasso - Mean Absolute Error (MAE) pour l'entraînement:", mae_train_lasso)
print("Lasso - Mean Absolute Error (MAE) pour le test:", mae_test_lasso)
print("Lasso - Root Mean Squared Error (RMSE) pour l'entraînement:", rmse_train_lasso)
print("Lasso - Root Mean Squared Error (RMSE) pour le test:", rmse_test_lasso)


SGD Regresor 

In [ ]:
sgd = SGDRegressor()
sgd.fit(X_train,y_train)
print("R2 train",sgd.score(X_train,y_train))
print("R2 test",sgd.score(X_test,y_test))
y_pred_sgd = sgd.predict(X_test)


print("rmse",np.sqrt(mean_squared_error(y_test, y_pred_sgd)))
print("mae", mean_absolute_error(y_test, y_pred_sgd))

sorted_y_test = np.sort(y_test)
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred_sgd)
plt.plot(sorted_y_test, sorted_y_test, color='red', linestyle='--') 
plt.xlabel("Valeurs réelles")
plt.ylabel("Prédictions")
plt.show();

AdaBoostRegressor

In [ ]:
ada = AdaBoostRegressor(n_estimators=100)
ada.fit(X_train,y_train)
print("R2 train",ada.score(X_train,y_train))
print("R2 test",ada.score(X_test,y_test))
y_pred_ada = ada.predict(X_test)


print("rmse",np.sqrt(mean_squared_error(y_test, y_pred_ada)))
print("mae", mean_absolute_error(y_test, y_pred_ada))

sorted_y_test = np.sort(y_test)
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred_ada)
plt.plot(sorted_y_test, sorted_y_test, color='red', linestyle='--') 
plt.xlabel("Valeurs réelles")
plt.ylabel("Prédictions")
plt.show();

GradientBoostingRegressor

In [ ]:
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=42)
gbr.fit(X_train, y_train)
y_pred_gbr = gbr.predict(X_test)
display(gbr.score(X_test, y_test))
display(gbr.score(X_train, y_train))

# Calculate Mean Absolute Error (MAE)
mae_gbr = mean_absolute_error(y_test, y_pred_gbr)

# Calculate Mean Squared Error (MSE) - vous l'avez déjà fait
mse_gbr = mean_squared_error(y_test, y_pred_gbr)

# Calculate Root Mean Squared Error (RMSE)
rmse_gbr = mean_squared_error(y_test, y_pred_gbr, squared=False)

# Calculate R-squared for test set
r2_gbr_test = r2_score(y_test, y_pred_gbr)

# Pour le jeu d'entraînement, vous devez d'abord prédire les valeurs pour X_train
y_pred_gbr_train = gbr.predict(X_train)

# Calculate R-squared for train set
r2_gbr_train = r2_score(y_train, y_pred_gbr_train)

# Afficher les résultats
print(f"MAE: {mae_gbr}")
print(f"MSE: {mse_gbr}")
print(f"RMSE: {rmse_gbr}")
print(f"R-squared train: {r2_gbr_train}")
print(f"R-squared test: {r2_gbr_test}")


XGBoost

In [ ]:
# modèle XG Boost sur données ICE + hyb


# Instancier le modèle XGBoostRegressor
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Entraîner le modèle
xgb_model.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred = xgb_model.predict(X_test)



# Calculer l'erreur quadratique moyenne (MSE) sur l'ensemble de test
mse = mean_squared_error(y_test, y_pred)

rmse = np.sqrt(mse)
              
R2_train = xgb_model.score(X_train,y_train)
R2_test  = xgb_model.score(X_test,y_test)

mae = np.mean(np.abs(y_test - y_pred))
print("Mean Absolute Error (MAE) :", mae)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared Train:", R2_train)
print("R-squared Test:", R2_test)




# Tri des valeurs de y_test pour tracer la droite y_pred = y_test
sorted_y_test = np.sort(y_test)

plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred)
#sns.scatterplot(x=yICE_test, y=yICE_pred,color='g')

plt.plot(sorted_y_test, sorted_y_test, color='red', linestyle='--')  # Droite y_pred = y_test
plt.xlabel("Valeurs réelles")
plt.ylabel("Prédictions")
plt.show()


# interpretation du modèle
explainer = shap.Explainer(xgb_model)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test)
shap.summary_plot(shap_values, X_test, plot_type="bar")

In [ ]:
# modèle XG Boost sur données ICE  


# Instancier le modèle XGBoostRegressor
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Entraîner le modèle
xgb_model.fit(X_thermique_train_scaled, y_thermique_train)

# Faire des prédictions sur l'ensemble de test
yICE_pred = xgb_model.predict(X_thermique_test_scaled)



# Calculer l'erreur quadratique moyenne (MSE) sur l'ensemble de test
mse = mean_squared_error(y_thermique_test, yICE_pred)

rmse = np.sqrt(mse)
              
R2_train = xgb_model.score(X_thermique_train_scaled,y_thermique_train)
R2_test  = xgb_model.score(X_thermique_test_scaled,y_thermique_test)

mae = np.mean(np.abs(y_thermique_test - yICE_pred))
print("Mean Absolute Error (MAE) :", mae)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared Train:", R2_train)
print("R-squared Test:", R2_test)



# Tri des valeurs de y_test pour tracer la droite y_pred = y_test
sorted_yICE_test = np.sort(y_thermique_test)

plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_thermique_test, y=yICE_pred)
#sns.scatterplot(x=yICE_test, y=yICE_pred,color='g')

plt.plot(sorted_yICE_test, sorted_yICE_test, color='red', linestyle='--')  # Droite y_pred = y_test
plt.xlabel("Valeurs réelles")
plt.ylabel("Prédictions")
plt.show()

# interpretation du modèle

explainer = shap.Explainer(xgb_model)
shap_values = explainer.shap_values(X_thermique_test_scaled)
shap.summary_plot(shap_values, X_thermique_test_scaled)
shap.summary_plot(shap_values, X_thermique_test_scaled, plot_type="bar")


DecisionTreeRegressor

In [ ]:
#Dataset complet 

tree = DecisionTreeRegressor(random_state=42)
tree.fit(X_train, y_train)

y_train_t_pred = tree.predict(X_train)
y_t_pred = tree.predict(X_test)
tree_train = tree.score(X_train, y_train)
tree_test = tree.score(X_test, y_test)

print("score train",tree_train)
print("score test", tree_test)

mae_test = mean_absolute_error(y_test, y_t_pred)

mse_test = mean_squared_error(y_test, y_t_pred)

rmse_test = mean_squared_error(y_test, y_t_pred, squared=False)

print("mae:", mae_test)
print("mse:",mse_test)
print("rmse:",rmse_test)

#Représentation graphique des valeurs prédites par rapport aux valeurs réelles 
plt.figure(figsize=(6, 4))
sns.scatterplot(x=y_test, y=y_t_pred, color='blue', label='Valeurs Prédites')

# Ajout d'une ligne d'identité
identity_line = np.linspace(min(y_test), max(y_test), 100)
plt.plot(identity_line, identity_line, '--', color='red', label='Ligne d\'Identité')

# Ajouter des étiquettes et une légende
plt.xlabel('Valeurs Réelles (y_test)')
plt.ylabel('Valeurs Prédites (y_pred)')
plt.title('Nuage de points: y_test vs y_pred')
plt.legend()
plt.show()



In [ ]:
# Obtenir l'importance des features
importance = tree.feature_importances_

# Créer un dataframe avec les noms des features et leur importance
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importance})

# Trier le dataframe par importance décroissante
feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False)

# Afficher les 10 features les plus importantes
print(feature_importance_df.head(10))

# Afficher l'importance des features sous forme de graphique à barres
plt.bar(range(len(feature_importance_df)), feature_importance_df['Importance'], align='center')
plt.xticks(range(len(feature_importance_df)), feature_importance_df['Feature'], rotation=90)
plt.ylabel('Importance')
plt.title('Feature Importance')
plt.show()



In [ ]:
# Arbres de décision pour df_thermique
tree = DecisionTreeRegressor(random_state=42)
tree.fit(X_thermique_train_scaled, y_thermique_train)


y_thermique_t_pred = tree.predict(X_thermique_test_scaled)
tree_train2 = tree.score(X_thermique_train_scaled, y_thermique_train)
tree_test2 = tree.score(X_thermique_test_scaled, y_thermique_test)

print("score train:",tree_train2)
print("score test:",tree_test2)

mae_test_t = mean_absolute_error(y_thermique_test, y_thermique_t_pred)

mse_test_t = mean_squared_error(y_thermique_test, y_thermique_t_pred)

rmse_test_t = mean_squared_error(y_thermique_test, y_thermique_t_pred, squared=False)

print("mae:", mae_test_t)
print("mse:", mse_test_t)
print("rmse:", rmse_test_t)

#Représentation graphique valeurs prédite vs valeurs réelles
plt.figure(figsize=(6, 4))
sns.scatterplot(x=y_thermique_test, y=y_thermique_t_pred, color='blue', label='Valeurs Prédites')

# Ajout d'une ligne d'identité
identity_line = np.linspace(min(y_thermique_test), max(y_thermique_test), 100)
plt.plot(identity_line, identity_line, '--', color='red', label='Ligne d\'Identité')

# Ajouter des étiquettes et une légende
plt.xlabel('Valeurs Réelles (y_thermique_test)')
plt.ylabel('Valeurs Prédites (y_thermique_pred)')
plt.title('Nuage de points: y_test vs y_pred voitures thermiques')
plt.legend()
plt.show()



In [ ]:
# Obtenir l'importance des features
importance = tree.feature_importances_

# Créer un dataframe avec les noms des features et leur importance
feature_importance_df_t = pd.DataFrame({'Feature': X_thermique.columns, 'Importance': importance})

# Trier le dataframe par importance décroissante
feature_importance_df_t = feature_importance_df_t.sort_values('Importance', ascending=False)

# Afficher les 10 features les plus importantes
print(feature_importance_df_t.head(10))

# Afficher l'importance des features sous forme de graphique à barres
plt.bar(range(len(feature_importance_df_t)), feature_importance_df_t['Importance'], align='center')
plt.xticks(range(len(feature_importance_df_t)), feature_importance_df_t['Feature'], rotation=90)
plt.ylabel('Importance')
plt.title('Feature Importance sans les hybrides')
plt.show()


RandomForestRegressor

In [ ]:
#  Random_forest   sur thermique + Hybrides



rf_model = RandomForestRegressor(n_estimators=100,random_state=42)
# Entraînez le modèle sur l'ensemble d'entraînement
rf_model.fit(X_train, y_train)
importances =rf_model.feature_importances_
# Afficher les importances des variables
for i, importance in enumerate(importances):
    print(f"Variable {i}: Importance = {importance}")

# Faites des prédictions sur l'ensemble de test
y_pred = rf_model.predict(X_test)
ICE_hyb_mse = mean_squared_error(y_test, y_pred)

R2_train = rf_model.score(X_train,y_train)
R2_test  = rf_model.score(X_test,y_test)

print('taille de X_test',X_test.shape)



rmse = np.sqrt(ICE_hyb_mse)

mae = np.mean(np.abs(y_test - y_pred))
print("Mean Absolute Error (MAE) :", mae)
print("Mean Squared Error:", ICE_hyb_mse)
print("Root Mean Squared Error:", rmse)
print("R-squared train:", R2_train)
print("R-squared test:", R2_test) 


In [ ]:
#  Random_forest sur thermique uniquement

rf_model_ICE = RandomForestRegressor(n_estimators=100,random_state=42)
# Entraînez le modèle sur l'ensemble d'entraînement
rf_model_ICE.fit(X_thermique_train, y_thermique_train)
Importance_Thermique =rf_model_ICE.feature_importances_

# Afficher les importances des variables
for i, importance in enumerate(Importance_Thermique):
    print(f"Variable {i}: Importance_Thermique = {importance}")

# Faites des prédictions sur l'ensemble de test
yICE_pred = rf_model_ICE.predict(X_thermique_test_scaled)
ICE_mse = mean_squared_error(y_thermique_test, yICE_pred)

 
R2_train_ICE = rf_model_ICE.score(X_thermique_train,y_thermique_train)
R2_test_ICE = rf_model_ICE.score(X_thermique_test,y_thermique_test)

print('taille de X_thermique_test',X_thermique_test_scaled.shape)



rmse = np.sqrt(ICE_mse)

mae = np.mean(np.abs(y_thermique_test - yICE_pred))
print("Mean Absolute Error (MAE) :", mae)
print("Mean Squared Error:", ICE_mse)
print("Root Mean Squared Error:", rmse)
print("R-squared train:", R2_train_ICE)
print("R-squared test:", R2_test_ICE)


In [ ]:
# Créer un graphique à barres pour visualiser les importances des variables
plt.figure(figsize=(15, 6))
plt.bar(range(len(importances)), importances, tick_label=X_train.columns,label='thermique+Hyb',color='blue')

plt.xlabel('Variables')
plt.ylabel('Importance')
plt.title('Importances des variables du modèle RandomForest thermique+Hyb')
plt.legend()
plt.xticks(rotation=45)
plt.show()


# Créer un graphique à barres pour visualiser les importances des variables
plt.figure(figsize=(15, 6))
plt.bar(range(len(Importance_Thermique)), Importance_Thermique, tick_label=X_thermique_train_scaled.columns,label='thermique',color='green')

plt.xlabel('Variables')
plt.ylabel('Importance')
plt.title('Importances des variables du modèle RandomForest thermique')
plt.xticks(rotation=45)
plt.legend()
plt.show()


In [ ]:
# Tri des valeurs de y_test pour tracer la droite y_pred = y_test
sorted_y_test = np.sort(y_test)

plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred,label='ICE+Hyb',color='b')
sns.scatterplot(x=y_thermique_test, y=yICE_pred,label='ICE',color='g')

plt.plot(sorted_y_test, sorted_y_test,color='red', linestyle='--')  # Droite y_pred = y_test
plt.xlabel("Valeurs réelles")
plt.ylabel("Prédictions")
plt.legend()
plt.title('Comparaison des prédictions du modèle RandomForest entre datasets thermique et thermique+Hyb ')
plt.show()
